In [5]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.datasets import cifar10

train_ds,test_ds = cifar10.load_data()

split = int(train_ds[0].shape[0] * 0.8)

train_ds,val_ds = (train_ds[0][:split],train_ds[1][:split]),(train_ds[0][split:],train_ds[1][split:])

data_augumentation = tf.keras.Sequential([
    tfl.Rescaling(1/255),
    tfl.RandomFlip(),
    tfl.RandomBrightness(factor=0.2),
    tfl.RandomContrast(factor=(0.2,0.9)),
    tfl.RandomZoom(height_factor=(0.1,0.3))
])

def preprocess(x,y):
    return data_augumentation(x),tf.one_hot(tf.squeeze(y),10)

train_ds = tf.data.Dataset.from_tensor_slices(train_ds).map(preprocess).batch(64)
val_ds = tf.data.Dataset.from_tensor_slices(val_ds).map(preprocess).batch(64)
test_ds = tf.data.Dataset.from_tensor_slices(test_ds).map(preprocess).batch(64)

base_model = tf.keras.applications.VGG19(include_top=False,weights="imagenet",input_shape=(32,32,3))
base_model.summary()

for layer in base_model.layers[:-5]:
    layer.trainable = False
    
base_model.summary()

model = tf.keras.Sequential([
    base_model,
    tfl.Flatten(),
    tfl.Dense(256,"relu"),
    tfl.Dense(128,"relu"),
    tfl.Dense(64,"relu"),
    tfl.Dense(10,"softmax"),
])

model.summary()

model.compile(
    optimizer="adam",
    loss= tf.keras.losses.categorical_crossentropy,
    metrics=["accuracy"]
)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1,
    callbacks=[
        tf.keras.callbacks.TensorBoard(log_dir="logs")
    ]
)

%load_ext tensorboard
%tensorboard --logdir logs

loss,accuracy = model.evaluate(test_ds)
print("Loss : ",loss)
print("Accuracy : ",accuracy)

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

157/157 [==============================] - 81s 517ms/step - loss: 2.3027 - accuracy: 0.1000
Loss :  2.3026561737060547
Accuracy :  0.10000000149011612
